<a href="https://colab.research.google.com/github/0Kyumy0/Machine_Learning_CodingDojo/blob/Supervised_models/preprocessingExercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

In [ ]:
filename = '/content/drive/MyDrive/DataScience/BasesDeDatos/insurance.csv'
df = pd.read_csv(filename)
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


# Part 1

How well can charges be predicted based on the patient's age, sex, BMI, number of children, smoking status, and region?

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


## Características
* Numérico: Age, bmi charges
* Ordinal: children
* Nominal: region, sex, smoker

In [ ]:
df['children'].value_counts()

0    574
1    324
2    240
3    157
4     25
5     18
Name: children, dtype: int64

* 18 a 24 años ->  0
* 25 a 34 años ->  1
* 35 a 44 años ->  2
* 45 a 54 años ->  3
* 55 a 64 años ->  4

In [ ]:
rangos = [(18, 24), (25, 34), (35, 44), (45, 54), (55, 64)]
reemplazos = [0, 1, 2, 3, 4]

#  I apply the replacement on the 'age' column of the DataFrame before doing the procedure
# in order to keep it in age ranges
for rango, reemplazo in zip(rangos, reemplazos):
    df['age'] = np.where((df['age'] >= rango[0]) & (df['age'] <= rango[1]), reemplazo, df['age'])

print(df)

      age     sex     bmi  children smoker     region      charges
0       0  female  27.900         0    yes  southwest  16884.92400
1       0    male  33.770         1     no  southeast   1725.55230
2       1    male  33.000         3     no  southeast   4449.46200
3       1    male  22.705         0     no  northwest  21984.47061
4       1    male  28.880         0     no  northwest   3866.85520
...   ...     ...     ...       ...    ...        ...          ...
1333    3    male  30.970         3     no  northwest  10600.54830
1334    0  female  31.920         0     no  northeast   2205.98080
1335    0  female  36.850         0     no  southeast   1629.83350
1336    0  female  25.800         0     no  southwest   2007.94500
1337    4  female  29.070         0    yes  northwest  29141.36030

[1338 rows x 7 columns]


In [ ]:
df['age'].value_counts()

3    287
0    278
1    271
2    260
4    242
Name: age, dtype: int64

In [ ]:
y = df['charges']
X = df.drop(columns = 'charges')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
cat_selector = make_column_selector(dtype_include='object')

train_cat_data = X_train[cat_selector(X_train)]
test_cat_data = X_test[cat_selector(X_test)]

In [ ]:
cat_selector(X_train) # I verify that the selected data is correct

['sex', 'smoker', 'region']

In [ ]:
ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

ohe_encoder.fit(train_cat_data)

train_ohe = ohe_encoder.transform(train_cat_data)
test_ohe = ohe_encoder.transform(test_cat_data)

In [ ]:
ohe_column_names = ohe_encoder.get_feature_names_out(train_cat_data.columns) # I get the names of the columns

train_ohe = pd.DataFrame(train_ohe, columns = ohe_column_names)
test_ohe = pd.DataFrame(test_ohe, columns = ohe_column_names)

## Numerical data standardization

In [ ]:
num_selector = make_column_selector(dtype_include='float') # I select the columns with numeric values

train_nums = X_train[num_selector(X_train)].reset_index(drop = True)
test_nums = X_test[num_selector(X_test)].reset_index(drop = True)

In [ ]:
scaler = StandardScaler()

scaler.fit(train_nums)

train_scaled = scaler.transform(train_nums)
test_scaled = scaler.transform(test_nums)
train_scaled

array([[-1.14087456],
       [-0.66584152],
       [ 1.52879447],
       ...,
       [-0.8879673 ],
       [ 2.84324666],
       [-0.10179179]])

In [ ]:
# I transform them into a dataframe
train_nums_scaled = pd.DataFrame(train_scaled, columns=train_nums.columns)
test_nums_scaled = pd.DataFrame(test_scaled, columns=test_nums.columns)

train_nums_scaled.head()

,bmi
0,-1.140875
1,-0.665842
2,1.528794
3,0.926476
4,-0.268178


In [ ]:
X_train_age = X_train['age'].reset_index(drop = True)
X_train_children = X_train['children'].reset_index(drop = True)

X_test_age = X_test['age'].reset_index(drop = True)
X_test_children = X_test['children'].reset_index(drop = True)

In [ ]:
X_train_processed = pd.concat([X_train_age, train_nums_scaled, X_train_children, train_ohe], axis=1) # join all of the data to make one DF (for test and train)
X_test_processed = pd.concat([X_test_age, test_nums_scaled, X_test_children, test_ohe], axis=1)

In [ ]:
X_train_processed

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,0,-1.140875,0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1,-0.665842,2,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,3,1.528794,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3,3,0.926476,1,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,3,-0.268178,2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
998,0,0.139468,4,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
999,2,-1.105101,5,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1000,4,-0.887967,0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1001,2,2.843247,2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


# Part 2

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
reg = LinearRegression()

In [ ]:
reg.fit(X_train_processed, y_train) # the training data is adjusted

LinearRegression()

In [ ]:
train_preds = reg.predict(X_train_processed) # I get the predictions from the processed data of the features
test_preds = reg.predict(X_test_processed)

In [ ]:
r2_train = r2_score(y_train, train_preds)
r2_test = r2_score(y_test, test_preds)
print(r2_train)
print(r2_test)

0.7402544710903217
0.7621668026554833


It can be seen that both data have a fairly high percentage reaching approximately 75%.
Similarly, the test data exceeds the training data by 2%, which is a good sign.